# Test code

In [1]:
# Community simulator package
from IPython.display import Image
from community_simulator import *
from community_simulator.usertools import *
from community_simulator.visualization import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends import backend_pdf as bpdf
import numpy as np
import scipy as sp
import pickle
colors = sns.color_palette()
%matplotlib inline

# Community selection package
from community_selection import *
from community_selection.A_experiment_functions import *
from community_selection.B_community_phenotypes import *
from community_selection.C_selection_algorithms import *
from community_selection.D_migration_algorithms import *
from community_selection.usertools import *

In [31]:
x = [1,2,3,4]
y = {"a":1, "b":2}
z = {"x":x, "y":y}
z
     
with open("data/test/test.p", "wb") as f:
    pickle.dump(z, f)
with open("data/test/test.p", "rb") as f:
    print(pickle.load(f))


{'x': [1, 2, 3, 4], 'y': {'a': 1, 'b': 2}}


In [2]:
# Make dynanmics by default we will use the microbial consumer resource model
def dNdt(N,R,params):
    return MakeConsumerDynamics(assumptions)(N,R,params)
def dRdt(N,R,params):
    return MakeResourceDynamics(assumptions)(N,R,params)
dynamics = [dNdt,dRdt]

# Global parameters
## Default parameters from community-simulator
## !!!Don't touch this dictionary!!!
assumptions = a_default.copy() # Start with default parameters

## Update parameters for community-selection
assumptions.update({
    'SA': 2100 * np.ones(1), #Number of species in each specialist family (here, 3 families of 60 species)
    'MA': 90 * np.ones(1), #Number of resources in each class
    'Sgen': 0, #Number of generalist species (unbiased sampling over alll resource classes)
    "n_wells": 12,
    "m": 0, # Mortality
    "scale": 10**6,  #scale is a conversion factor specifying the number of individual microbial cells present when N = 1.
    "sigma" : 1, # Standard deviation for drawing specifc speices/interaction function
    "alpha": 1, # Scaling factor between species- and interaction-specific function variances
    "l": 0, # Set leakage function to 0 to switch off cross-feeding
    "response": "type III",
    "sigma_max": 5,
    'R0_food': 1000, # Total amount of supplied food
    "rich_medium": True, # Number of food types passed to R0
    "binary_threshold": 1,  
    # The parameters below will be passed to params_simulation
    "n_propagation": 1, # Length of propagation, or hours within a growth cycle
    "n_transfer": 2, # Number of total transfer, or number of passage
    "n_transfer_selection": 1, # Number of transfer implementing seleciton regimes
    "dilution": 1/1000, # Dilution factor at every transfer
    "n_inoc": 10**6,  #Number of cells sampled from the regional species at start
    "n_migration": 10**6, # Number of cells to be migrated in the migration perturbation algorithm
    "R_percent": 0, # Fracion of new resources to be spiked in to the media in the resource perturbation algorithm
    "selected_function": "f1_additive"
})

# Prepare experiment setup in this universe
seed_temp = 1
params, params_simulation = prepare_experiment(assumptions, seed = seed_temp)

In [3]:
data_directory = "data/test/"
list_algorithms = ["simple_screening"]
i = 0
algorithms = make_algorithms(params_simulation)


In [4]:
assumptions = assumptions
params = params
dynamics = dynamics
params_simulation = params_simulation
params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]]
write_composition = True
write_function = True
file_name = data_directory + "SP" + str(seed_temp) + "-" + list_algorithms[i]
assembly_type = str(list_algorithms[i])

In [5]:
# Set seeds
np.random.seed(2)

# Make initial state
init_state = MakeInitialState(assumptions)

# Make plate
plate = Community(init_state, dynamics, params, scale = assumptions["scale"], parallel = True) 

# Update the community composition by sampling from the pool
print("\nGenerating initial plate")
plate.N = sample_from_pool(plate.N, scale = assumptions["scale"], inocula = params_simulation["n_inoc"])

# Update the supplied resource if assumptions["rich_medium"]
if assumptions["rich_medium"]:
    plate.R = make_rich_medium(plate.R, assumptions)
    plate.R0 = make_rich_medium(plate.R, assumptions) # R0 for refreshing media on passaging if "refresh_resoruce" is turned on

# Add the attributes that are essential to the function measurement to the plate objects 
print("\nAdding attributes that are essential to the community function to the plate object")
plate = add_community_function(plate, dynamics, assumptions, params, params_simulation, params_algorithm)

# Empty list for saving data
plate_data_list = list() # Plate composition
community_function_list = list() # Community function

# Save the inocula composition
plate_data = reshape_plate_data(plate, transfer_loop_index = 0, assembly_type = assembly_type, community_function_name = params_algorithm["community_phenotype"][0]) # Initial state
plate_data_list.append(plate_data)

# Save the initial function
community_function = globals()[params_algorithm["community_phenotype"][0]](plate, assumptions = assumptions) # Community phenotype
richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
biomass = list(np.sum(plate.N, axis = 0)) # Biomass
function_data = reshape_function_data(community_function_name = params_algorithm["community_phenotype"][0], community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = 0, assembly_type = assembly_type)        
community_function_list.append(function_data) # Transfer = 0 means that it's before selection regime works upon


# Output the plate composition and community functions if write_composition set True
plate_data.to_csv(file_name + "-" + params_algorithm["community_phenotype"][0] + "-T" + "{:02d}".format(0) + "-composition.txt", index = False)
function_data.to_csv(file_name + "-" + params_algorithm["community_phenotype"][0] + "-T" + "{:02d}".format(0) + "-function.txt", index = False)

print("\nStart propogation")
# Run simulation
for i in range(0, params_simulation["n_transfer"]):
    # Algorithms used in this transfer
    phenotype_algorithm = params_algorithm["community_phenotype"][i]
    selection_algorithm = params_algorithm["selection_algorithm"][i]
    migration_algorithm = params_algorithm["migration_algorithm"][i]

    # Print the propagation progress
    if (i % 5) == 0:
        print("Transfer " + str(i+1))

    # Propagation
    plate.Propagate(params_simulation["n_propagation"])

    # Append the composition to a list
    plate_data = reshape_plate_data(plate, transfer_loop_index = i + 1, assembly_type = assembly_type, community_function_name = phenotype_algorithm) # Transfer = 0 means that it's before selection regime works upon
    plate_data_list.append(plate_data)

    # Community phenotype, richness, and biomass
    community_function = globals()[phenotype_algorithm](plate, assumptions = assumptions) # Community phenotype
    richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
    biomass = list(np.sum(plate.N, axis = 0)) # Biomass
    function_data = reshape_function_data(community_function_name = phenotype_algorithm, community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = i + 1 , assembly_type = assembly_type)        
    community_function_list.append(function_data) # Transfer = 0 means that it's before selection regime works upon

    # Output the plate composition and community functions if write_composition set True
    if ((i+1) == assumptions["n_transfer_selection"]) or ((i+1) == assumptions["n_transfer"]):
            plate_data.to_csv(file_name + "-" + phenotype_algorithm + "-T" + "{:02d}".format(i + 1) + "-composition.txt", index = False) # Transfer = 0 means that it's before selection regime works upon
    
    function_data.to_csv(file_name + "-" + phenotype_algorithm + "-T" + "{:02d}".format(i + 1) + "-function.txt", index = False)

    # Save the plate object    
    if (i+1) == assumptions["n_transfer_selection"]:
        with open("data/test/test.p", "wb") as f:
            pickle.dump({"plate_N":plate.N, "plate_R":plate.R}, f)

    # Passage and tranfer matrix
    transfer_matrix = globals()[selection_algorithm](community_function)
    plate.Passage(transfer_matrix * params_simulation["dilution"])


    # Migration
    m = globals()[migration_algorithm](community_function) 
    plate.N = migrate_from_pool(plate, migration_factor = m, scale = assumptions["scale"], inocula = params_simulation["n_migration"]) # By default, n_migration is the same as n_inoc

print("\nAlgorithm "+ params_algorithm["algorithm_name"][0] + " finished")



Generating initial plate

Adding attributes that are essential to the community function to the plate object

Start propogation
Transfer 1

Algorithm simple_screening finished


In [1]:
import pickle
plate1 = pickle.load(open("data/test/test.p", "rb"))
plate1

{'plate_N':                 W0        W1        W2         W3         W4         W5  \
 F0 S0     0.000000  0.000000  0.000000   1.907832   0.000000   0.000000   
    S1     0.000000  0.000000  0.000000   0.000000   0.000000   0.000000   
    S2     0.000000  0.000000  0.000000   0.422468   0.000000   0.000000   
    S3     0.000000  0.000000  0.000000   0.000000   0.000924   0.000000   
    S4     0.000000  0.000000  0.000000   0.000000   0.000000   0.000000   
    S5     0.000000  0.000000  0.000000   0.000000   0.000000   0.000000   
    S6     0.000000  0.000000  0.000000   0.678898   0.000000   0.000000   
    S7     0.000000  0.000000  0.000000   0.000000   0.000000   0.000000   
    S8     0.000000  0.000000  0.000000   0.000000   0.000000   0.000000   
    S9     0.000000  0.000000  0.000000   0.000000   0.000000   0.000000   
    S10    0.000000  0.000000  0.000000   0.000000   0.000000   0.000000   
    S11    0.000000  0.000000  0.000000   0.000000   0.000000   0.000000   
 

In [3]:
data_directory = "data/test/"
list_algorithms = ["simple_screening"]

# Parameters for simulation
params_simulation.update({"selected_function": "f1_additive"}) # selected function

# Make the list of algorithms
algorithms = make_algorithms(params_simulation)

# Simulation
for i in range(len(list_algorithms)):
    simulate_community(
        assumptions = assumptions,
        params = params,
        dynamics = dynamics,
        params_simulation = params_simulation,
        params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]],
        write_composition = True,
        write_function = True,
        file_name = data_directory + "SP" + str(seed_temp) + "-" + list_algorithms[i],
        assembly_type = str(list_algorithms[i]),
    )


Algorithm: simple_screening


 transfer community_phenotype selection_algorithm migration_algorithm
        1         f1_additive        no_selection        no_migration
        2         f1_additive        no_selection        no_migration

Generating initial plate

Adding attributes that are essential to the community function to the plate object

Start propogation
Transfer 1

Algorithm simple_screening finished


In [3]:
#list_algorithms = ["Jochum2019", "Panke_Buisse2015", "Penn2004", "Xie2019a", "Xie2019b", "Arora2019_V2",  "Arora2019_V2_control", "Raynaud2019a_V2", "Raynaud2019a_V2_control", "Raynaud2019b_V2", "Raynaud2019b_V2_control"]
list_algorithms = ["resource_old", "resource", "resource_add", "resource_remove", "resource_rescale_add", "resource_rescale_remove"]

algorithms = make_algorithms(params_simulation)

[i in list(algorithms["algorithm_name"]) for i in list_algorithms]


[True, True, True, True, True, True]

In [5]:
data_directory = "data/test/"
list_phenotypes = ["f1_additive"]
#list_algorithms = ["knock_in_isolates", "knock_out"]
#list_algorithms = ["Jochum2019", "Panke_Buisse2015", "Penn2004", "Xie2019a", "Xie2019b", "Arora2019_V2",  "Arora2019_V2_control", "Raynaud2019a_V2", "Raynaud2019a_V2_control", "Raynaud2019b_V2", "Raynaud2019b_V2_control"]
#list_algorithms = ["Arora2019_V2",  "Arora2019_V2_control", "Raynaud2019a_V2", "Raynaud2019a_V2_control", "Raynaud2019b_V2", "Raynaud2019b_V2_control"]
#list_algorithms = ["resource_old", "resource", "resource_add", "resource_remove", "resource_rescale_add", "resource_rescale_remove", "knock_in_isolates", "knock_out"]
#list_algorithms = ["knock_in_isolates", "knock_out"]
#list_algorithms = ["Swenson2000b_control"]
list_algorithms = ["simple_screening"]


for j in range(len(list_phenotypes)):
    # Parameters for simulation
    params_simulation.update({"selected_function": list_phenotypes[j]}) # selected function

    # Make the list of algorithms
    algorithms = make_algorithms(params_simulation)

    # Simulation
    for i in range(len(list_algorithms)):
        simulate_community(
            assumptions = assumptions,
            params = params,
            dynamics = dynamics,
            params_simulation = params_simulation,
            params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]],
            write_composition = True,
            write_function = True,
            file_name = data_directory + "SP" + str(seed_temp) + "-" + list_algorithms[i],
            assembly_type = str(list_algorithms[i]),
        )


Algorithm: Swenson2000b_control


 transfer community_phenotype          selection_algorithm migration_algorithm
        1         f1_additive  select_top25percent_control        no_migration
        2         f1_additive  select_top25percent_control        no_migration
        3         f1_additive  select_top25percent_control        no_migration
        4         f1_additive  select_top25percent_control        no_migration
        5         f1_additive  select_top25percent_control        no_migration
        6         f1_additive                 no_selection        no_migration
        7         f1_additive                 no_selection        no_migration
        8         f1_additive                 no_selection        no_migration
        9         f1_additive                 no_selection        no_migration
       10         f1_additive                 no_selection        no_migration

Generating initial plate

Adding attributes that are essential to the community function to the

In [57]:
def sample_from_pool_richness(plate_N, scale = 10**6):
    S_tot = plate_N.shape[0] # Total number of species in the pool
    
    # Manipulated inocula sizes    
    inocula_size = list()
    for i in range(7):
        inocula_size = inocula_size + [2**i]*20
        
    consumer_index = plate_N.index # Consumer index
    well_names = ["W" + str(i) for i in range(len(inocula_size))] # Well index

    N0 = np.zeros((plate_N.shape[0], len(inocula_size))) # Make empty plate
    
    for k in range(len(inocula_size)):
        pool = np.random.power(0.01, size = S_tot) # Power-law distribution
        pool = pool/np.sum(pool) # Normalize the pool
        consumer_list = np.random.choice(S_tot, size = inocula_size[k], replace = True, p = pool) # Draw from the pool
        my_tab = pd.crosstab(index = consumer_list, columns = "count") # Calculate the cell count
        N0[my_tab.index.values,k] = np.ravel(my_tab.values / scale) # Scale to biomass

    # Make data.frame
    N0 = pd.DataFrame(N0, index = consumer_index, columns = well_names)
    N0.iloc[2097,:] = 1/scale

    return N0
plate_N = sample_from_pool_richness(plate.N)

In [65]:
#np.where(plate_N["W100"] != 0)[0]
plate.params

{'c': array([[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'm': 0,
 'w': 1,
 'D': array([[6.91848627e-09, 3.42699214e-02, 9.47622418e-04, ...,
         1.24306759e-03, 7.39294934e-08, 1.10549397e-16],
        [1.03404403e-04, 9.12509363e-25, 4.18152740e-07, ...,
         1.18390647e-07, 1.10046827e-07, 2.11210973e-11],
        [5.26695465e-08, 5.01881338e-04, 1.45151994e-31, ...,
         6.15803006e-09, 1.25029923e-10, 1.72980565e-08],
        ...,
        [8.05465214e-10, 6.49315980e-08, 9.62934891e-11, ...,
         1.66371098e-08, 1.18463709e-32, 4.25952635e-07],
        [2.47689089e-21, 5.71469839e-03, 6.85089552e-03, ...,
         2.52103155e-05, 8.70427928e-07, 3.67026989e-02],
        [4.42949341e-07, 2.19011871e-06, 4.03597137e-52, ...,
         1.85809562e-27, 1.24989638e-10, 1.77196

In [60]:
# Print out the algorithms
print("\nAlgorithm: "+ params_algorithm["algorithm_name"][0])
print("\n")
print(params_algorithm[["transfer", "community_phenotype", "selection_algorithm", "migration_algorithm"]].to_string(index = False))

# Set seeds
np.random.seed(2)

# Make initial state
init_state = MakeInitialState(assumptions)

# Make plate
plate = Community(init_state, dynamics, params, scale = assumptions["scale"], parallel = True) 

# Update the community composition by sampling from the pool
print("\nGenerating initial plate")
plate.N = sample_from_pool_richness(plate.N, scale = assumptions["scale"])
plate

# Update the supplied resource if assumptions["rich_medium"]
if assumptions["rich_medium"]:
    plate.R = make_rich_medium(plate.R, assumptions)
    plate.R0 = make_rich_medium(plate.R, assumptions) # R0 for refreshing media on passaging if "refresh_resoruce" is turned on

# Add the attributes that are essential to the function measurement to the plate objects 
print("\nAdding attributes that are essential to the community function to the plate object")
plate = add_community_function(plate, dynamics, assumptions, params, params_simulation, params_algorithm)

# Empty list for saving data
plate_data_list = list() # Plate composition
community_function_list = list() # Community function

# Save the inocula composition
plate_data = reshape_plate_data(plate, transfer_loop_index = 0, assembly_type = assembly_type, community_function_name = params_algorithm["community_phenotype"][0]) # Initial state
plate_data_list.append(plate_data)

# Save the initial function
community_function = globals()[params_algorithm["community_phenotype"][0]](plate, assumptions = assumptions) # Community phenotype
richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
biomass = list(np.sum(plate.N, axis = 0)) # Biomass
function_data = reshape_function_data(community_function_name = params_algorithm["community_phenotype"][0], community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = 0, assembly_type = assembly_type)        
community_function_list.append(function_data) # Transfer = 0 means that it's before selection regime works upon



Algorithm: simple_screening


 transfer community_phenotype selection_algorithm migration_algorithm
        1   f5_invader_growth        no_selection        no_migration
        2   f5_invader_growth        no_selection        no_migration
        3   f5_invader_growth        no_selection        no_migration
        4   f5_invader_growth        no_selection        no_migration
        5   f5_invader_growth        no_selection        no_migration
        6   f5_invader_growth        no_selection        no_migration
        7   f5_invader_growth        no_selection        no_migration
        8   f5_invader_growth        no_selection        no_migration
        9   f5_invader_growth        no_selection        no_migration
       10   f5_invader_growth        no_selection        no_migration

Generating initial plate

Adding attributes that are essential to the community function to the plate object
Sampling invader (resident) community

Stabilizing the invader (resident) community. Pass

ValueError: shapes (90,10) and (140,140) not aligned: 10 (dim 1) != 140 (dim 0)

In [5]:
community_function

W0    1.000083
W1    1.000083
W2    1.000083
W3    1.000083
W4    1.000083
W5    1.000083
W6    1.000083
W7    1.000083
W8    1.000083
W9    1.000083
dtype: float64